# Leonardo's Mechanical Lion: Automata & Programming

> *"The body of the animal is a mechanical system..."*  
> — Leonardo da Vinci

## Introduction
In 1515, Leonardo built a mechanical lion to entertain the King of France. It could walk, stop, and open its chest to reveal fleurs-de-lis. This was an early example of a **programmable robot**.

In this notebook, we explore:
1.  **Cam Programming**: Using shaped disks to control complex motion.
2.  **Leg Kinematics**: Converting rotary motion to walking steps.
3.  **Stability**: Keeping the lion from tipping over.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Cam Profiles
A cam converts rotary motion into linear motion. The shape of the cam determines the movement pattern of the leg (lift, swing, plant).
$$ r(\theta) = R_{base} + h(\theta) $$

### 2. Stability (Support Polygon)
For the lion to stay upright, its Center of Mass (COM) must remain inside the polygon formed by the feet touching the ground.
$$ COM \in \text{ConvexHull}(P_{feet}) $$


In [ ]:
def generate_cam_profile(lift_height, dwell_angle_deg):
    theta = np.linspace(0, 360, 360)
    radius = np.ones_like(theta) * 10.0 # Base radius 10cm
    
    # Create a simple lift profile
    # Dwell (ground contact) -> Lift -> Swing -> Plant
    
    dwell_start = 0
    dwell_end = dwell_angle_deg
    
    for i, t in enumerate(theta):
        if t < dwell_end:
            # Stance phase (constant radius)
            radius[i] = 10.0
        else:
            # Swing phase (lift)
            # Normalized swing progress (0 to 1)
            swing_prog = (t - dwell_end) / (360 - dwell_end)
            # Sinusoidal lift
            lift = lift_height * np.sin(swing_prog * np.pi)
            radius[i] = 10.0 + lift
            
    return theta, radius

def plot_lion_mechanics(lift, dwell):
    theta, radius = generate_cam_profile(lift, dwell)
    
    fig = plt.figure(figsize=(12, 5))
    
    # Plot 1: Cam Profile (Polar)
    ax1 = fig.add_subplot(121, projection='polar')
    ax1.plot(np.radians(theta), radius, 'k-', linewidth=2)
    ax1.fill(np.radians(theta), radius, alpha=0.1, color='gray')
    ax1.set_title('Leg Control Cam')
    ax1.set_ylim(0, 20)
    
    # Plot 2: Foot Path (Cartesian)
    ax2 = fig.add_subplot(122)
    # Simplified conversion of cam radius to foot height
    foot_height = radius - 10.0
    stride_progress = theta / 360.0
    
    ax2.plot(stride_progress, foot_height, 'b-', linewidth=2)
    ax2.fill_between(stride_progress, 0, foot_height, alpha=0.1, color='blue')
    ax2.set_title('Foot Height over One Cycle')
    ax2.set_xlabel('Cycle Progress')
    ax2.set_ylabel('Height (cm)')
    ax2.set_ylim(0, 10)
    
    plt.tight_layout()
    plt.show()
    
    stance_percent = (dwell / 360) * 100
    print(f"Stance Phase: {stance_percent:.1f}% (Ground Contact)")
    print(f"Swing Phase: {100-stance_percent:.1f}% (Leg Lifted)")
    if stance_percent < 50:
        print("WARNING: Stance phase < 50%. Lion may be unstable!")

interact(plot_lion_mechanics, 
         lift=widgets.FloatSlider(min=1.0, max=8.0, step=0.5, value=4.0, description='Lift (cm)'),
         dwell=widgets.IntSlider(min=90, max=270, step=10, value=200, description='Stance Angle'));

## Conclusion

The shape of the cam acts as the **software** for the lion. By changing the cam profile, Leonardo could program different gaits (walking, trotting). 

For stability, at least 3 legs usually need to be on the ground at once (static stability), implying a stance phase of >75% for each leg, or careful synchronization of diagonal pairs.